In [196]:
%matplotlib inline

In [208]:
from scipy.optimize import brentq
from tabulate import tabulate
import numpy as np
from scipy.optimize import linprog
import pprint

days_in_year = 364.
bonds = [
    { # PIK ISIN ru000A1016z3 https://smart-lab.ru/q/bonds/RU000A0ZZZU3 returns 8.1%
        'N': 1000.,
        'P': 1005,
        'g': 0.0821,
        'p': 4., # changed to simplify
        'q': 45.,
        'T': 4.49,
        'n': 0,
        'external_returns': 8.1,
        'ISIN': 'ru000A1016z3',
        'irr': 0.0,
        'D': 0.0,
    },
    { # SoftLine ISIN RU000A0ZZZU3 https://smart-lab.ru/q/bonds/RU000A0ZZZU3 returns 9.9%
        'N': 1000.,
        'P': 1020,
        'g': 0.11,
        'p': 2.,
        'q': 0.0,
        'T': 613./days_in_year,
        'n': 0.0,
        'external_returns': 9.9,
        'ISIN': 'RU000A0ZZZU3',
        'irr': 0.0,
        'D': 0.0,
    },
    { # Норильский никель ISIN RU000A100VQ6 https://smart-lab.ru/q/bonds/RU000A100VQ6 returns 6.79% 
        'N': 1000.,
        'P': 1009.7,
        'g': 0.072,
        'p': 2,
        'q': 0.0,
        'T': 4.28,
        'n': 0.0,
        'external_returns': 6.79,
        'ISIN': 'RU000A100VQ6',
        'irr': 0.0,
        'D': 0.0,
    }
]

def fill_in_derivatives(bond):
    bond['q'] = bond['g'] * bond['N'] / bond['p']
    bond['n'] = int(bond['T'] * bond['p'])
    if bond['T'] * bond['p'] - bond['n'] != 0:
        bond['n'] += 1
        

for bond in bonds:
    fill_in_derivatives(bond)


def calculate_irr(x):
    n = bond['n']
    tau = n / bond['p'] - bond['T']

    return bond['P'] - (1 + x)**tau * (bond['g'] * bond['N'] / bond['p']\
            * (1 - (1 + x)**(-n/bond['p']))/((1 + x)**(1/bond['p']) - 1) + bond['N'] / (1 + x)**(n / bond['p']))


def calculate_pv(cf_i: float,
                 returns: float,
                 t_i: float):
    return cf_i/(1+returns)**t_i


def calculate_pv_compound(bonds:dict,
                          t_i: float,
                          CF_i: list,
                          bond_exhausted:bool=False):
    PV_CF_i = [.0]* len(bonds)
    for e in range(len(bonds)):
        b = bonds[e]
        #print(f"##### exhausted_nominals[e] {exhausted_nominals[e]} e {e}")
        bond_time_delta = 1. / b['p']
        #print(f"##### bond_time_delta {bond_time_delta} t_i {t_i}")
        if not bond_exhausted and (t_i % bond_time_delta) > 0.001:
            CF_i[e] = .0
        returns = b['g']
        #print(exhausted_nominals[e])
        PV_CF_i[e] = CF_i[e] /(1.+returns)**t_i # per bond PV_CF_i
    return PV_CF_i

def calculate_cf_i_compound(bonds:list, t_i:float, bond_exhausted:bool=False):
    result = []
    time_boundaries = get_timeboundaries(bonds)
    for b in bonds:
        CF_i_bond = .0
        #print(f'!!!!!!! {t_i % float(k)} t_i {t_i} float(k) {float(k)}')
        time_delta = 1. / b['p']
        if bond_exhausted or (t_i % time_delta) < 0.00001:
        #print(f'######### key {float(k)} time moment {t_i} r {bond["compound_g"][k]}')
            CF_i_bond = b['k'] * b['N'] * b['g'] / b['p']
        result.append(CF_i_bond)
    return result

def calculate_p_compound(bonds:list):
    result = []
    for b in bonds:
        result.append(b['k'] * b['P'])
    return result
    
def get_timeboundaries(bonds: list):
    time_boundaries = []
    for b in bonds:
        time_boundaries.append(b['n'])
    return time_boundaries

def print_calculation_table(bond: dict, output:bool=False):
    P = bond['P']
    returns = bond['g']
    N = bond['N']
    delta_t = 1. / bond['p']
    time_boundaries = None
    if bond.get('compound_g', None) is not None:
        time_boundaries = get_timeboundaries(bond['bonds'])
    tabulate_rows = []
    tabulate_headers = ['№','t_i', 'CF_i', 'PV(CF_i)', 'D PV(CF_i)/P', 'PV(CF_i)/P*t_i', 'C t_i*(t_i+1)*PV(CF_i)/P']
    for i in range(1, bond['n']+1):
        CF_i_list = []
        t_i = delta_t * i
        CF_i = .0
        idx = -1
        PV_CF_i = .0
        P_list = []
        PV_CF_i_div_P = .0
        dur = .0
        conv_coef = .0
        if bond.get('compound_g', None) is not None: # don't check for an empty dict here
            bond_exhausted = i in time_boundaries
            exhausted_nominals = [.0]*len(bond['bonds'])
            exhausted_bond_nominal = .0
            CF_i_list = calculate_cf_i_compound(bond['bonds'], t_i, bond_exhausted)
            if bond_exhausted is True: # might be two bonds ending in the same t_i
                idx = time_boundaries.index(i) # bond index
                b = bond['bonds'][idx]
                #print(f"%%%%%%%%%%% {idx} exhausted {b}")
                #print(f"{CF_i_list[idx]}")
                CF_i_list[idx] += b['N'] * b['k'] # get original bond nominal price
                #print(f"{CF_i_list[idx]}")
            PV_CF_list = calculate_pv_compound(bond['bonds'], t_i, CF_i_list, bond_exhausted) # dup CF_i calc
            P_list = calculate_p_compound(bond['bonds'])
            for e in PV_CF_list:
                PV_CF_i += e
            for c in CF_i_list:
                CF_i += c
            for e in range(len(CF_i_list)):
                PV_CF_i_div_P += PV_CF_list[e] / P_list[e]
                dur += PV_CF_list[e] / P_list[e] * t_i
                conv_coef += PV_CF_list[e] / P_list[e] * t_i * (t_i +1)
            if idx > 0:
                del(bond['bonds'][idx])
                del(time_boundaries[idx])
            tabulate_rows.append([i, 
                          t_i, 
                          CF_i, 
                          PV_CF_i, 
                          PV_CF_i_div_P,
                          dur, 
                          conv_coef])
        else:
            CF_i = returns * N / bond['p']
            if i == bond['n']:
                CF_i += N
            PV_CF_i = calculate_pv(CF_i, returns, t_i)
            tabulate_rows.append([i, 
                                  t_i, 
                                  CF_i, 
                                  PV_CF_i, 
                                  PV_CF_i / P,
                                  PV_CF_i / P * t_i, 
                                  PV_CF_i / P * t_i * (t_i +1)])
    result_array = np.array(tabulate_rows)
    # Aggregates 6 - duration, 7 convex coeff
    sum_row_initial = [None, None, None] + list(np.sum(tabulate_rows, axis=0, dtype=float)[3:])
    #print(sum_row_initial)
    bond['D'] = sum_row_initial[5]
    tabulate_rows.append(sum_row_initial)
    if output is True:
        print(tabulate(tabulate_rows, headers=tabulate_headers, floatfmt=".8f"))
    print("\n")
    return result_array


def construct_compound_bond(bonds:list = None):
    if bonds is None:
        return
    result = {'N': .0,
              'P': .0,
              'g': .0,
              'compound_g': [],
              'compound_N': [],
              'compound_delta_i': [],
              'compound_n': [],
              'P': .0,
              'p': .0,
              'irr': .0,
              'n': .0,
              'g_delta_i': {},
              'bonds': [],
              'exausted_bonds': [],
    }
    
    for b in bonds:
        if b['k'] < 0:
            continue
        result['N'] += b['N']
        result['P'] += b['P']
        result['g'] += b['g'] # Incorrect value
        result['compound_g'].append(b['g']) # Calculation for CF_i
        result['compound_N'].append(b['N'])
        result['P'] = max(b['P'], result['P'])
        result['p'] = max(b['p'], result['p'])
        result['compound_delta_i'].append(1. / b['p'])
        result['irr'] = max(b['irr'], result['irr'])
        result['n'] = max(b['n'], result['n'])
        result['bonds'].append(b)

    compound_g = {}
    compound_N = {}
    for e in range(0, len(result['compound_delta_i'])):
        k = str(result['compound_delta_i'][e])
        compound_g[k] = compound_g.get(k, .0) + result['compound_g'][e]
        #print(bonds[e]['k'])
        compound_N[k] = compound_N.get(k, .0) + result['compound_N'][e] * bonds[e]['k']
    #print(compound_N)
    result['compound_g'] = compound_g
    result['compound_N'] = compound_N
    return result

   
bonds_irr_dur = {}
# Ищем корень нелинейного уравнения
for bond in bonds:
    # internal rr for the bond
    root = brentq(calculate_irr, 0.01, 0.4)
    bond['irr'] = root
    print(f"ISIN {bond['ISIN']} Доверенное значение доходности {bond['external_returns']}% расчетная внутренняя доходность {root*100:.3f}%")
    print_calculation_table(bond, output=False)
    #print(bond)

i_horizon = 3.0
V = 1000000.
r_0 = 0.055
r_1 = 0.045
r_2 = 0.05
T_1 = i_horizon
T_2 = 1.

dur = [bond['D'] for bond in bonds]
          
A_eq = [[dur[0], dur[1], dur[2]], [1., 1., 1.]]
B_eq = [i_horizon, 1, ]

# portfolio fractions
x_1 = np.linalg.lstsq(A_eq, B_eq, rcond=None)[0]
for e in range(0,3):
    bonds[e]['x'] = x_1[e]
    bonds[e]['k'] = int(x_1[e] * V / bonds[e]['N'])
ratio = []

print(f"Initial ratio in the portfolio")
for bond in bonds:
    if bond['x'] > 0:
        print(f"{bond['ISIN']} : {bond['x']}")
        
          
       
pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(bonds)
compound_bond = construct_compound_bond(bonds)
pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(compound_bond)
print("Compound bond statistics.")
result_array = print_calculation_table(compound_bond, True)

# Planned return on investment
V_0_r_0_3 = V * (1+r_0)**3.0
print(f"Planned return on investement at t_0 with returns rate {r_0} {V_0_r_0}")
# Actual return on investement
min_time_delta = 255.
for b in compound_bond['bonds']:
    min_time_delta = min(1. / b['p'], min_time_delta)
n = T_1 / min_time_delta

V_0_r_1_3 = .0
returns_column = result_array[:,2]
counter = 0
for r in returns_column:
    if counter < n:
        V_0_r_1_3 += r * (1 + r_1) ** (T - min_time_delta * n)
    else:
        V_0_r_1_3 += r / (1 + r_1) ** (min_time_delta * n - T)
    counter += 1
print(f"Actual return on investement with safe returns rate changes from {r_0} down to {r_1} {V_0_r_1_3}")
print(f"Portfolio is safe against returns rates change from {r_0} to {r_1}")

cash = .0
time_passed = (T_1 - T_2) / min_time_delta
for e in range(int(time_passed)):
    cash += returns_column[e]
portfolio_cost = .0
time_left = len(returns_column) - (T_1 - T_2) / min_time_delta
for e in range(int(time_left)):
    portfolio_cost += returns_column[int(time_passed)+e] / (1 + r_1)**e

print(f"At moment T = {T_1 - T_2} we have {cash} in cash and the portfolio costs {portfolio_cost}.")



ISIN ru000A1016z3 Доверенное значение доходности 8.1% расчетная внутренняя доходность 8.347%


ISIN RU000A0ZZZU3 Доверенное значение доходности 9.9% расчетная внутренняя доходность 12.326%


ISIN RU000A100VQ6 Доверенное значение доходности 6.79% расчетная внутренняя доходность 7.501%


Initial ratio in the portfolio
ru000A1016z3 : 0.2908178306990571
RU000A0ZZZU3 : 0.42357028804686614
RU000A100VQ6 : 0.2856118812540768
  №         t_i             CF_i          PV(CF_i)    D PV(CF_i)/P    PV(CF_i)/P*t_i    C t_i*(t_i+1)*PV(CF_i)/P
---  ----------  ---------------  ----------------  --------------  ----------------  --------------------------
  1  0.25000000    5952.25000000     5835.98698152      0.02002397        0.00500599                  0.00625749
  2  0.50000000   39477.25000000    37713.61517930      0.10524894        0.05262447                  0.07893671
  3  0.75000000    5952.25000000     5610.22935050      0.01924937        0.01443703                  0.02526480
  4  1.0000000